In [1]:
import os
import sys
import torch
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from torchvision import models
from torch import nn as nn
import numpy as np

from dataset.MicroscopyTrainDataLoader import MicroscopyTrainDataLoader
from torch.utils.data import DataLoader

In [2]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = nn.Sequential(*list(model.children())[:-2])
model.eval()
img = torch.Tensor(2, 3, 224, 224).normal_() 
outputs = model(img)
print(outputs.shape)

#outputs = outputs.view(-1)
#print(outputs.shape)

torch.Size([2, 2048, 7, 7])


In [3]:
CSV_PATH = '../labels/microscopy.csv'
loader = MicroscopyTrainDataLoader(CSV_PATH)
training_dataset = loader.get_train_dataset(augmentation=False)
validation_dataset = loader.get_val_dataset()

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

kwargs = {'num_workers': 16, 'pin_memory': True} if torch.cuda.is_available() else {}
train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True, **kwargs)


y_true = []
x = []
for images, labels, img_path in train_loader:
    y_true += labels
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    outputs = outputs.flatten(start_dim=1)
    outputs = outputs.detach().cpu().numpy()
    x = np.vstack([outputs]) if x == [] else np.vstack([x, outputs])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [5]:
x.shape

(1862, 100352)

In [6]:
len(y_true)

1862

In [7]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(x, y_true)

SVC(kernel='linear')

In [8]:
test_dataset = loader.get_test_dataset()
kwargs = {'num_workers': 16, 'pin_memory': True} if torch.cuda.is_available() else {}
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, **kwargs)

In [9]:
y_test = []
y_pred = []
x_test = []
for images, labels, img_path in test_loader:
    y_test += labels
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    outputs = outputs.flatten(start_dim=1)
    outputs = outputs.detach().cpu().numpy()
    x_test = np.vstack([outputs]) if x_test == [] else np.vstack([x_test, outputs])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [10]:
clf.score(x_test, y_test)

0.8579610538373424

In [11]:
y_true = clf.predict(x_test)

In [18]:
from sklearn.metrics import classification_report
target_names = test_dataset.codec.classes_
print(classification_report(np.array(y_test), y_true, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        DMEL     0.5441    0.4205    0.4744        88
        DMFL     0.9107    0.9331    0.9217       284
        DMLI     0.9074    0.9679    0.9367       405
        DMTR     0.6707    0.5729    0.6180        96

    accuracy                         0.8580       873
   macro avg     0.7582    0.7236    0.7377       873
weighted avg     0.8458    0.8580    0.8502       873



In [16]:
y_true

array([0, 1, 1, 1, 3, 1, 0, 1, 0, 1, 0, 1, 0, 3, 0, 3, 0, 2, 3, 1, 0, 0,
       1, 2, 0, 2, 0, 3, 0, 1, 0, 1, 2, 1, 2, 3, 2, 0, 2, 0, 2, 0, 2, 3,
       2, 0, 2, 3, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0,
       0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 3,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,